In [ ]:
# Facial Emotion Recognition Model - Optimized for Snapdragon X Elite
# =======================================================================
# Configuration: SCRATCH with optimized hyperparameters
# Target: <1 hour training time with maximum accuracy

import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image, UnidentifiedImageError

# Import TensorFlow and configure for optimal performance
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"Python executable: {sys.executable}")

import seaborn as sns

# Enable mixed precision for faster training on Snapdragon X Elite
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print("Mixed precision training enabled - expect 2-3x speedup!")

# Import deep learning libraries
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Flatten, Dropout, Dense, Input, 
                                     GlobalAveragePooling2D, Conv2D, 
                                     BatchNormalization, Activation, MaxPooling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications import 

# =======================================================================
# Configuration Parameters
# =======================================================================

picture_size = 48  # Input image size (48x48 grayscale)
batch_size = 128
epochs = 30
no_of_classes = 7  # Angry, Disgust, Fear, Happy, Neutral, Sad, Surprise
learning_rate = 0.0001

print(f"Image size: {picture_size}x{picture_size} grayscale")
print(f"Batch size: {batch_size}")
print(f"Epochs: {epochs}")

# Auto-detect folder path
if sys.executable.startswith('/anaconda/envs/azureml_py38_PT_TF/bin/python'):
    folder_path = "Users/martyn.frank/IATD_Deeplearning/Project/data/images/"
else:
    folder_path = 'data/images/'
    
print(f"Data folder path: {folder_path}")

# =======================================================================
# Image Validation and Cleaning
# =======================================================================

def is_image_valid(filepath):
    """Validate image integrity."""
    try:
        with Image.open(filepath) as img:
            img.verify()
        return True
    except (UnidentifiedImageError, OSError):
        return False

def delete_if_corrupt(filepath):
    """Delete corrupted images."""
    try:
        with Image.open(filepath) as img:
            img.verify()
        return False
    except (UnidentifiedImageError, OSError):
        print(f"Deleting corrupted image: {filepath}")
        os.remove(filepath)
        return True


TensorFlow version: 2.19.1
Python executable: c:\Users\marty\anaconda3\envs\tf311_env\python.exe
Mixed precision training enabled - expect 2-3x speedup!
Image size: 48x48 grayscale
Batch size: 128
Epochs: 30
Data folder path: data/images/
Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Training samples: 28821
Validation samples: 7066
Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 12, 12, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,02

 Total params: 4,478,727 (17.08 MB)

 Trainable params: 4,474,759 (17.07 MB)

 Non-trainable params: 3,968 (15.50 KB)

In [5]:

# =======================================================================
# Advanced Image Preprocessing
# =======================================================================

def preprocess_image(image):
    # If image comes as float, convert to uint8 for CLAHE
    if image.dtype != 'uint8':
        image = (image * 255).astype('uint8')
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
    # ---- CHECK CHANNELS ----
    if len(image.shape) == 2:  # Grayscale (H,W)
        enhanced = clahe.apply(image)
        enhanced = enhanced[..., np.newaxis]  # (H,W,1)
    elif len(image.shape) == 3 and image.shape[2] == 1:  # Grayscale (H,W,1)
        enhanced = clahe.apply(image.squeeze())
        enhanced = enhanced[..., np.newaxis]  # (H,W,1)
    elif len(image.shape) == 3 and image.shape[2] == 3:  # RGB (H,W,3)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        enhanced = clahe.apply(gray)
        enhanced = enhanced[..., np.newaxis]  # (H,W,1)
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    # Normalize to [0, 1]
    enhanced = enhanced.astype('float32') / 255.0
    return enhanced

class PreprocessingImageDataGenerator(ImageDataGenerator):
    """Custom generator with advanced preprocessing."""
    
    def __init__(self, *args, preprocessing_function=None, **kwargs):
        super().__init__(*args, preprocessing_function=preprocessing_function, **kwargs)
    
    def standardize(self, x):
        x = super().standardize(x)
        # Apply additional preprocessing
        return preprocess_image(x)


In [6]:

# =======================================================================
# Data Augmentation and Loading
# =======================================================================

# Training data generator with CLAHE preprocessing
datagen_train = PreprocessingImageDataGenerator(
    rescale=1./255,
    rotation_range=20,              # Increased rotation
    width_shift_range=0.15,          # Increased shift range
    height_shift_range=0.15,
    shear_range=0.15,                # Add shear transformation
    zoom_range=0.15,                 # Increased zoom
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],     # Brightness variation
    fill_mode='nearest'
)

datagen_validation = ImageDataGenerator(rescale=1./255)

# Create training set
train_set = datagen_train.flow_from_directory(
    folder_path + "train/",
    target_size=(picture_size, picture_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Create validation set
test_set = datagen_validation.flow_from_directory(
    folder_path + "validation/",
    target_size=(picture_size, picture_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print(f"Training samples: {train_set.n}")
print(f"Validation samples: {test_set.n}")
print(f"Class indices: {train_set.class_indices}")

# =======================================================================
# Model Architecture - Custom CNN from Scratch
# =======================================================================

# Building CNN from scratch (baseline)
model = Sequential([
    # Block 1
    Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 2
    Conv2D(128, (5, 5), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 3
    Conv2D(512, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 4
    Conv2D(512, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Dense layers
    Flatten(),
    Dense(256),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(512),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(no_of_classes, activation='softmax')
])

# Compile the model
opt = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Training samples: 28821
Validation samples: 7066
Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 12, 12, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 256)            │         1,02

 Total params: 4,478,727 (17.08 MB)

 Trainable params: 4,474,759 (17.07 MB)

 Non-trainable params: 3,968 (15.50 KB)

In [7]:

# =======================================================================
# Training Callbacks
# =======================================================================

checkpoint = ModelCheckpoint(
    filepath="best_model.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,  # Increased patience
    verbose=1,
    restore_best_weights=True
)

reduce_learningrate = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_delta=0.0001,
    min_lr=0.00001
)

callbacks_list = [checkpoint, early_stopping, reduce_learningrate]


In [8]:

# =======================================================================
# Model Training
# =======================================================================

print("Starting training...")
print(f"Target: {epochs} epochs with early stopping")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")
print("Mixed precision: ENABLED")

history = model.fit(
    train_set,
    steps_per_epoch=train_set.n // train_set.batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.n // test_set.batch_size,
    callbacks=callbacks_list,
    verbose=1
)


Starting training...
Target: 30 epochs with early stopping
Batch size: 128
Learning rate: 0.0001
Mixed precision: ENABLED


c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 19/225 ━━━━━━━━━━━━━━━━━━━━ 10:12:39 178s/step - accuracy: 0.1509 - loss: 2.7231

KeyboardInterrupt: 

In [ ]:

# Save the final trained model

# Base filename for model
import re
base_name = 'final_model'
extension = '.keras'

# List all files in the current directory matching the pattern
existing_files = [f for f in os.listdir('.') if re.match(rf'{base_name}_v\d+{extension}', f)]

# Find the highest version number
if existing_files:
    version_numbers = [
        int(re.search(rf'{base_name}_v(\d+){extension}', fname).group(1))
        for fname in existing_files
    ]
    next_version = max(version_numbers) + 1
else:
    next_version = 1

# Create new filename
new_filename = f"{base_name}_v{next_version}{extension}"

# Save model
model.save(new_filename)
print(f"Model saved as: {new_filename}")print("✅ Model training complete!")


In [ ]:

# =======================================================================
# Training Visualization
# =======================================================================

plt.style.use('dark_background')
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss', fontsize=14)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.title('Model Accuracy', fontsize=14)
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

# =======================================================================
# Model Evaluation
# =======================================================================

print("\n" + "="*70)
train_loss, train_acc = model.evaluate(train_set, verbose=0)
test_loss, test_acc = model.evaluate(test_set, verbose=0)

print(f"📊 FINAL RESULTS")
print(f"Training Accuracy: {train_acc*100:.2f}%")
print(f"Validation Accuracy: {test_acc*100:.2f}%")
print(f"Training Loss: {train_loss:.4f}")
print(f"Validation Loss: {test_loss:.4f}")
print("="*70)

# =======================================================================
# Detailed Performance Analysis
# =======================================================================

from sklearn.metrics import confusion_matrix, classification_report
from keras.models import load_model

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Load best model
my_model = load_model('best_model.keras', compile=False)

# Generate predictions on validation set
print("Generating predictions on validation set...")
predictions = []
true_labels = []

for i in range(len(test_set)):
    batch_images, batch_labels = test_set[i]
    batch_predictions = my_model.predict(batch_images, verbose=0)
    predictions.extend(np.argmax(batch_predictions, axis=1))
    true_labels.extend(np.argmax(batch_labels, axis=1))
    if i >= test_set.n // test_set.batch_size:
        break

predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='rocket', 
            xticklabels=class_labels, 
            yticklabels=class_labels,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Facial Emotion Recognition', fontsize=16, pad=20)
plt.xlabel('Predicted Emotion', fontsize=12)
plt.ylabel('True Emotion', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification Report
print("\n" + "="*70)
print("📈 DETAILED CLASSIFICATION METRICS")
print("="*70)
print(classification_report(true_labels, predictions, target_names=class_labels))

# =======================================================================
# Export to ONNX for Snapdragon Optimization
# =======================================================================

try:
    import tf2onnx
    import onnx
    
    print("\n" + "="*70)
    print("🚀 Exporting model to ONNX format for Snapdragon optimization...")
    
    # Convert to ONNX
    spec = (tf.TensorSpec((None, 48, 48, 1), tf.float32, name="input"),)
    output_path = "emotion_model.onnx"
    
    model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
    
    print(f"✅ Model exported to {output_path}")
    print("This model is optimized for Snapdragon X Elite inference!")
    print("="*70)
    
except ImportError:
    print("⚠️ tf2onnx not installed. Run: pip install tf2onnx")
except Exception as e:
    print(f"⚠️ ONNX export failed: {e}")

# =======================================================================
# Save Model Weights
# =======================================================================

model.save_weights('emotion_model_weights.h5')
print("\n✅ Model weights saved successfully!")
print("\n🎉 Training complete! Your optimized model is ready.")
print(f"⏱️ Expected training time: 30-50 minutes on Snapdragon X Elite")